### Have some extra sessions from Mike, haven't had object_features behavior files produced yet, see what the good pairs look like with these sessions included

In [17]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import utils.behavioral_utils as behavioral_utils
import utils.information_utils as information_utils
import utils.visualization_utils as visualization_utils
import utils.glm_utils as glm_utils
from matplotlib import pyplot as plt
import utils.spike_utils as spike_utils
from constants.glm_constants import *
from constants.behavioral_constants import *

import seaborn as sns
import scipy.stats


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
num_bins = 2

def get_sess_beh(row, beh_path, sub):
    session = row.session_name
    behavior_path = beh_path.format(sess_name=session)
    beh = pd.read_csv(behavior_path)

    # # filter trials 
    # beh = behavioral_utils.get_valid_trials(beh)
    beh = beh[beh.Response.isin(["Correct", "Incorrect"])]
    feature_selections = behavioral_utils.get_selection_features(beh)
    beh = pd.merge(beh, feature_selections, on="TrialNumber", how="inner")
    beh["session"] = session
    return beh

In [24]:
sub = "BL"
# Monkey B
# all_units = pd.read_pickle("/data/patrick_res/firing_rates/BL/all_units.pickle")
# sessions = pd.DataFrame({"session_name": all_units.session.unique()})
sessions = pd.read_pickle("/data/patrick_res/sessions/BL/valid_sessions.pickle")
all_beh = pd.concat(sessions.apply(lambda x: get_sess_beh(x, BL_SESS_BEHAVIOR_PATH, "BL"), axis=1).values).reset_index()

In [27]:
all_beh[all_beh.BlockNumber == 0].groupby("CurrentRule").session.nunique()

CurrentRule
ESCHER     1
RIPPLE    60
Name: session, dtype: int64

In [20]:
all_beh

,index,TrialNumber,BlockNumber,TrialAfterRuleChange,TaskInterrupt,ConditionNumber,Response,ItemChosen,TrialType,CurrentRule,...,Item2Shape,Item2Color,Item2Pattern,Item3Shape,Item3Color,Item3Pattern,Color,Shape,Pattern,session
0,0,27,2,0,NaN,3698,Incorrect,2.0,6,YELLOW,...,SQUARE,MAGENTA,POLKADOT,STAR,GREEN,RIPPLE,MAGENTA,SQUARE,POLKADOT,20190128
1,1,28,2,1,NaN,3871,Incorrect,2.0,6,YELLOW,...,TRIANGLE,MAGENTA,RIPPLE,SQUARE,GREEN,POLKADOT,MAGENTA,TRIANGLE,RIPPLE,20190128
2,2,29,2,2,NaN,3975,Incorrect,1.0,6,YELLOW,...,STAR,MAGENTA,SWIRL,TRIANGLE,GREEN,POLKADOT,CYAN,CIRCLE,RIPPLE,20190128
3,3,30,2,3,NaN,3852,Incorrect,2.0,6,YELLOW,...,CIRCLE,MAGENTA,RIPPLE,SQUARE,GREEN,SWIRL,MAGENTA,CIRCLE,RIPPLE,20190128
4,4,31,2,4,NaN,3891,Incorrect,1.0,6,YELLOW,...,CIRCLE,MAGENTA,SWIRL,TRIANGLE,GREEN,RIPPLE,CYAN,STAR,ESCHER,20190128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39352,88,150,3,81,NaN,3829,Correct,0.0,6,YELLOW,...,SQUARE,MAGENTA,RIPPLE,TRIANGLE,GREEN,POLKADOT,YELLOW,STAR,ESCHER,20190123
39353,89,151,3,82,NaN,3679,Correct,0.0,6,YELLOW,...,TRIANGLE,MAGENTA,RIPPLE,STAR,GREEN,POLKADOT,YELLOW,CIRCLE,ESCHER,20190123
39354,90,152,3,83,NaN,3504,Correct,0.0,6,YELLOW,...,CIRCLE,MAGENTA,ESCHER,STAR,GREEN,SWIRL,YELLOW,TRIANGLE,RIPPLE,20190123
39355,91,153,3,84,NaN,3984,Correct,0.0,6,YELLOW,...,TRIANGLE,MAGENTA,RIPPLE,STAR,GREEN,SWIRL,YELLOW,SQUARE,ESCHER,20190123


In [4]:
block_thresh = 3
pairs = behavioral_utils.get_good_pairs_across_sessions(all_beh, block_thresh)
pairs.sort_values(by="num_sessions", ascending=False)[:20]

,pair,sessions,num_sessions,dim_type
61,"[ESCHER, RIPPLE]","[20190529, 20190617, 20190710, 20190816, 20190...",5,within dim
62,"[ESCHER, SWIRL]","[20190529, 20190627, 20190814, 20190823]",4,within dim
65,"[RIPPLE, SWIRL]","[20190524, 20190529, 20190823]",3,within dim
27,"[STAR, POLKADOT]","[20190605, 20190606, 20190607]",3,across dim
2,"[CIRCLE, TRIANGLE]","[20190603, 20190606, 20190812]",3,within dim
34,"[TRIANGLE, ESCHER]","[20190221, 20190710, 20190814]",3,across dim
1,"[CIRCLE, STAR]","[20190228, 20190606, 20190703]",3,within dim
60,"[ESCHER, POLKADOT]","[20190227, 20190531, 20190816]",3,within dim
25,"[STAR, YELLOW]","[20190524, 20190605, 20190703]",3,across dim
52,"[MAGENTA, ESCHER]","[20190130, 20190530, 20190823]",3,across dim


### Try to read one of the CSVs

In [5]:
df = pd.read_csv("/data/patrick_res/scratch/Blanche-BehaviorOnly/sub-BL_sess-190125_parsedbehavior.csv", header=None)
df = df.rename(columns={0: "TrialNumber", 2: "BlockNumber", 3: "CurrentRule"})[["TrialNumber", "BlockNumber", "CurrentRule"]]

In [7]:
rule_map = [
    "", 
    "POLKADOT", "RIPPLE", "ESCHER", "SWIRL", 
    "CYAN", "MAGENTA", "YELLOW", "GREEN", 
    "CIRCLE", "TRIANGLE", "STAR", "SQUARE"
]

df["CurrentRule"] = df["CurrentRule"].apply(lambda x: rule_map[x])

In [8]:
df.CurrentRule.unique()

array(['CYAN', 'ESCHER', 'SWIRL', 'CIRCLE', 'STAR', 'GREEN', 'RIPPLE',
       'POLKADOT', 'SQUARE', 'MAGENTA'], dtype=object)

In [9]:
df

,TrialNumber,BlockNumber,CurrentRule
0,9,1,CYAN
1,10,1,CYAN
2,11,1,CYAN
3,12,1,CYAN
4,13,1,CYAN
...,...,...,...
540,550,13,CYAN
541,551,13,CYAN
542,552,13,CYAN
543,553,13,CYAN


### Aggregate all of em

In [28]:
base_dir = "/data/patrick_res/scratch/Blanche-BehaviorOnly/"
file_names = os.listdir(base_dir)


In [29]:
rule_map = [
    "", 
    "POLKADOT", "RIPPLE", "ESCHER", "SWIRL", 
    "CYAN", "MAGENTA", "YELLOW", "GREEN", 
    "CIRCLE", "TRIANGLE", "STAR", "SQUARE"
]
dfs = []
for file_name in file_names:
    if file_name.endswith(".csv"): 
        sess_name = "20" + file_name.split("_")[1].split("-")[1]
        df = pd.read_csv(os.path.join(base_dir, file_name), header=None)
        df = df.rename(columns={0: "TrialNumber", 2: "BlockNumber", 3: "CurrentRule"})[["TrialNumber", "BlockNumber", "CurrentRule"]]
        df["CurrentRule"] = df["CurrentRule"].apply(lambda x: rule_map[x])
        df["session"] = sess_name
        dfs.append(df)
new_beh = pd.concat(dfs)

In [30]:
new_beh[new_beh.BlockNumber == 1].groupby("CurrentRule").session.nunique()

CurrentRule
CYAN        38
ESCHER      15
MAGENTA      5
POLKADOT     3
SQUARE       1
STAR         5
SWIRL        5
TRIANGLE     9
YELLOW       8
Name: session, dtype: int64

In [15]:
block_thresh = 3
new_pairs = behavioral_utils.get_good_pairs_across_sessions(new_beh, block_thresh)
new_pairs = new_pairs.sort_values(by="num_sessions", ascending=False)[:20]

In [16]:
new_pairs

,pair,sessions,num_sessions,dim_type
62,"[ESCHER, SWIRL]","[20190225, 20190529, 20190530, 20190604, 20190...",12,within dim
65,"[RIPPLE, SWIRL]","[20190215, 20190524, 20190528, 20190529, 20190...",9,within dim
61,"[ESCHER, RIPPLE]","[20190529, 20190611, 20190617, 20190710, 20190...",9,within dim
41,"[CYAN, ESCHER]","[20190207, 20190220, 20190225, 20190529, 20190...",8,across dim
29,"[STAR, SWIRL]","[20190215, 20190524, 20190621, 20190625, 20190...",7,across dim
39,"[CYAN, MAGENTA]","[20190201, 20190228, 20190610, 20190708, 20190...",7,within dim
40,"[CYAN, YELLOW]","[20190225, 20190612, 20190617, 20190621, 20190...",7,within dim
60,"[ESCHER, POLKADOT]","[20190225, 20190227, 20190531, 20190605, 20190...",7,within dim
21,"[STAR, TRIANGLE]","[20190606, 20190624, 20190702, 20190703, 20190...",7,within dim
52,"[MAGENTA, ESCHER]","[20190130, 20190530, 20190604, 20190815, 20190...",6,across dim


In [61]:
old_beh = all_beh[["TrialNumber", "BlockNumber", "CurrentRule", "session"]]

In [62]:
agg_beh = pd.concat((old_beh, new_beh))

In [66]:
block_thresh = 3
agg_pairs = behavioral_utils.get_good_pairs_across_sessions(agg_beh, block_thresh)
agg_pairs = agg_pairs.sort_values(by="num_sessions", ascending=False)[:20]

In [71]:
good_pairs = agg_pairs[agg_pairs.num_sessions > 6]

In [96]:
len(good_pairs)

11

In [78]:
sessions = good_pairs.sessions.explode().unique()

In [109]:
len(sessions)

47

### Get sessions that have been spike sorted, sessions that have object_features..

In [83]:
all_units = pd.read_pickle("/data/patrick_res/firing_rates/BL/all_units.pickle")
sorted_sessions = all_units.session.unique()
of_sessions = pd.read_pickle("/data/patrick_res/sessions/BL/valid_sessions.pickle").session_name.unique()

### Get sessions where no object features have been generated yet

In [103]:
no_of_sessions = sessions[~np.isin(sessions, of_sessions)]
df = pd.DataFrame({"sessions": no_of_sessions})
df.to_csv("/data/patrick_res/scratch/no_of_sessions.csv")

### Get sessions where object features have been generated, but haven't been spike sorted yet

In [107]:
of_sessions = sessions[np.isin(sessions, of_sessions)]
of_no_spike_sess = of_sessions[~np.isin(of_sessions, sorted_sessions)]
df = pd.DataFrame({"sessions": of_no_spike_sess})
df.to_csv("/data/patrick_res/scratch/of_no_spike_sessions.csv")

In [108]:
good_pairs.to_csv("/data/patrick_res/scratch/good_pairs.csv")